In [3]:
import whisperx
import gc 

device = "cuda" 
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

model_id = "NbAiLabBeta/nb-whisper-small"
model = whisperx.load_model(model_id, device, compute_type=compute_type)

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.1.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/torch/whisperx-vad-segmentation.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.1+cu118. Bad things might happen unless you revert torch to 1.x.


In [6]:
import torch

audio_file = "store/audio.mp3"
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"])
# delete model if low on GPU resources
import gc; gc.collect(); torch.cuda.empty_cache(); del model

Detected language: no (0.96) in first 30s of audio...
[{'text': ' Hjerteskjermfrukt- eller jordfluer er en invasjon av gjerde i California. Jordfluer legger egg og hekker utrolig raskt. Men i dag og nesten hver dag skal millioner av fly slutte i California og utenfor Los Angeles. Og i Los Angeles-området spesifikt. Det er en veldig god grunn til dette.', 'start': 0.009, 'end': 21.886}, {'text': ' Californa opplever smitte fra 1987 til 1994. I 1994 begynte California en toårig test for å avgjøre om løslatelsen av sterile fly vil forhindre målinger som reduserte størrelsen på smittene. Under den toårige testperioden har California opplevd null smitte. I 1996 begynte medflytelsesprogrammet, og programmet nå er 1750 km2.', 'start': 21.886, 'end': 51.886}, {'text': ' I stedet for å bremse mannlige fluer på trygge områder fordi de er etablert, steriliserer fluene når de på universet med stråler og tagger dem med ultrafiolett. Det at de reduserer noe, gjør ikke det radioaktivt, men det gjør f

In [8]:
# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"]) # after alignment
import gc; gc.collect(); torch.cuda.empty_cache(); del model_a

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


model.safetensors:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Some weights of the model checkpoint at NbAiLab/nb-wav2vec2-1b-bokmaal were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at NbAiLab/nb-wav2vec2-1b-bokmaal and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this 

[{'start': 0.009, 'end': 4.613, 'text': ' Hjerteskjermfrukt- eller jordfluer er en invasjon av gjerde i California.', 'words': [{'word': 'Hjerteskjermfrukt-', 'start': 0.009, 'end': 1.33, 'score': 0.168}, {'word': 'eller', 'start': 1.35, 'end': 1.69, 'score': 0.281}, {'word': 'jordfluer', 'start': 1.71, 'end': 2.551, 'score': 0.162}, {'word': 'er', 'start': 2.591, 'end': 2.671, 'score': 0.382}, {'word': 'en', 'start': 2.691, 'end': 2.751, 'score': 0.749}, {'word': 'invasjon', 'start': 2.791, 'end': 3.091, 'score': 0.328}, {'word': 'av', 'start': 3.111, 'end': 3.252, 'score': 0.125}, {'word': 'gjerde', 'start': 3.272, 'end': 3.652, 'score': 0.031}, {'word': 'i', 'start': 3.672, 'end': 3.792, 'score': 0.18}, {'word': 'California.', 'start': 3.832, 'end': 4.613, 'score': 0.351}]}, {'start': 4.673, 'end': 7.655, 'text': 'Jordfluer legger egg og hekker utrolig raskt.', 'words': [{'word': 'Jordfluer', 'start': 4.673, 'end': 5.173, 'score': 0.108}, {'word': 'legger', 'start': 5.193, 'end': 5.

In [9]:
# load hf-token from secrets.yml
import yaml
secrets = yaml.load(open("secrets.yml", "r"))
HF = secrets["HF"]
diarize_model = whisperx.DiarizationPipeline(use_auth_token=HF, device=device)

# add min/max number of speakers if known
# diarize_segments = diarize_model(audio, min_speakers=1, max_speakers=2)
diarize_segments = diarize_model(audio)
result = whisperx.assign_word_speakers(diarize_segments, result)
print(diarize_segments)
print(result["segments"]) # segments are now assigned speaker IDs

config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

                              segment label     speaker       start  \
0   [ 00:00:00.008 -->  00:00:20.008]     A  SPEAKER_01    0.008489   
1   [ 00:00:20.008 -->  00:00:21.519]     B  SPEAKER_00   20.008489   
2   [ 00:00:21.926 -->  00:00:53.998]     C  SPEAKER_00   21.926995   
3   [ 00:00:53.998 -->  00:01:03.556]     D  SPEAKER_01   53.998302   
4   [ 00:01:03.675 -->  00:01:18.514]     E  SPEAKER_01   63.675722   
5   [ 00:01:12.317 -->  00:01:12.334]     F  SPEAKER_00   72.317487   
6   [ 00:01:19.057 -->  00:02:26.392]     G  SPEAKER_00   79.057725   
7   [ 00:02:26.629 -->  00:02:37.852]     H  SPEAKER_01  146.629881   
8   [ 00:02:38.310 -->  00:03:16.001]     I  SPEAKER_00  158.310696   
9   [ 00:03:16.001 -->  00:03:19.567]     J  SPEAKER_01  196.001698   
10  [ 00:03:19.787 -->  00:03:23.455]     K  SPEAKER_01  199.787776   
11  [ 00:03:23.981 -->  00:03:57.292]     L  SPEAKER_00  203.981324   
12  [ 00:03:57.716 -->  00:04:16.307]     M  SPEAKER_01  237.716469   
13  [ 